In [576]:
from keras import backend as K
from keras.engine.topology import Layer
from keras.models import Sequential
from keras.layers import Dense, Activation
import keras

from sklearn import datasets, model_selection, preprocessing, metrics

import numpy as np

import theano
import theano.tensor as T

import scipy
from scipy.fftpack import rfft, irfft


## Comparing numpy and scipy FFT

In [577]:
def numpy_fft(x):
    #numpy doesn't preserve dtype
    return np.fft.irfft(np.fft.rfft(x)*np.fft.rfft(x)).astype(np.float32)

def scipy_fft(x):
    #scipy preserves dtype
    return scipy.fftpack.irfft(scipy.fftpack.rfft(x)*scipy.fftpack.rfft(x))

In [578]:
x=np.random.random(size=(1000,1000)).astype(np.float32)

%timeit numpy_fft(x)
%timeit scipy_fft(x)

10 loops, best of 3: 68.1 ms per loop
10 loops, best of 3: 28.9 ms per loop


## Implementing circulant layer

### Theano forward and gradient operations 

In [522]:
class CirculantConvolutionGradientOp(theano.Op):
    def __init__(self):
        super(CirculantConvolutionGradientOp, self).__init__()

    def make_node(self, r, X, O):
        r = T.as_tensor_variable(r)
        X = T.as_tensor_variable(X)
        O = T.as_tensor_variable(O)

        return theano.Apply(self, [r, X, O], [T.vector(), T.matrix()])

    def perform(self, node, inputs, output_storage):
        
        r = inputs[0]
        X = inputs[1]        
        O = inputs[2]
        
        Offt = rfft(O)        

        Xfft = rfft(X)        
        XOconv = irfft(Xfft*Offt)
        grad_r = np.sum(XOconv, axis=0)
        
        r_flipped_shifted = np.roll(r[::-1],1)
        Rfft = rfft(r_flipped_shifted)
        grad_X = irfft(Rfft*Offt)

        output_storage[0][0] = grad_r
        output_storage[1][0] = grad_X

    def infer_shape(self, node, input_shapes):
        return [input_shapes[0], input_shapes[1]]

In [524]:
class CirculantConvolutionOp(theano.Op):
    def __init__(self):
        super(CirculantConvolutionOp, self).__init__()

    def make_node(self, r, X):
        r = T.as_tensor_variable(r)
        X = T.as_tensor_variable(X)
        
        assert r.ndim == 1
        assert X.ndim == 2
        
        return theano.Apply(self, [r, X], [T.matrix()])

    def perform(self, node, inputs, output_storage):
        r = inputs[0]
        X = inputs[1]
        
        Rfft = rfft(r)
        Xfft = rfft(X)
        
        result = irfft(Rfft*Xfft)
        
        z = output_storage[0]
        z[0] = result

    def infer_shape(self, node, input_shapes):
        return [input_shapes[1]]

    def grad(self, inputs, output_grads):
        r = inputs[0]
        X = inputs[1]
        O = output_grads[0]
        
        grad_op = CirculantConvolutionGradientOp()
        return grad_op(r,X, O)

In [574]:
r = T.dvector()
X = T.dmatrix()
conv_op = CirculantConvolutionOp()
f = theano.function([r, X], conv_op(r, X))

print(f([1,2], [[1,2],[1,1],[0,0]]))

[[ 5.  4.]
 [ 3.  3.]
 [ 0.  0.]]


In [575]:
T.verify_grad(conv_op,[np.array([1,2], dtype=np.float32), np.array([[1,2],[1,1],[0,0]], dtype=np.float32)],
                         rng=np.random.RandomState())

### Implementing linear layer with circulant weights

In [554]:
class CirculantLayer(Layer):

    def __init__(self, output_dim,  **kwargs):
        self.output_dim = output_dim
        super(CirculantLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.sign_flipping = np.random.randint(0,2, size = input_shape[1])
        self.sign_flipping = (self.sign_flipping*2-1).astype(np.float32)
        
        self.input_dim = input_shape[1]
        
        circulant_vec_shape = (self.input_dim,)
        if self.input_dim<self.output_dim:
            circulant_vec_shape = (self.output_dim,)
        
        self.circulant_vec = self.add_weight(name='circulant_vec', 
                                      shape=circulant_vec_shape,
                                      initializer='uniform',
                                      trainable=True)
        
        self.bias = self.add_weight(name='bias', 
                                      shape=(self.output_dim,),
                                      initializer='uniform',
                                      trainable=True)
        
        
        super(CirculantLayer, self).build(input_shape) 

    def call(self, X):
        X =  K.T.as_tensor_variable(self.sign_flipping) * X  
        
        if self.input_dim<self.output_dim:
            zeros = K.T.zeros((X.shape[0], self.output_dim))
            X = K.T.set_subtensor(zeros[:,:self.input_dim], X)
        
        r = self.circulant_vec

        conv_op = CirculantConvolutionOp()
        RX =conv_op(r, X)
        
        if self.input_dim>self.output_dim:
            RX = RX[:, :self.output_dim]
        
        return RX+self.bias

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

## Testing on digits dataset

In [555]:
digits = datasets.load_digits()
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
le = preprocessing.OneHotEncoder()
y = le.fit_transform(digits.target.reshape(-1, 1)).todense()

In [556]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.33, random_state=42)

In [557]:
def fit_circulant():
    model = Sequential()
    model.add(CirculantLayer(1000, input_shape=[X.shape[1]]))
    model.add(Activation('relu'))
    model.add(CirculantLayer(1000))
    model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation('sigmoid'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, verbose = 0)
    return model

In [551]:
def fit_dense():
    model = Sequential()
    model.add(Dense(1000, input_shape=[X.shape[1]]))
    model.add(Activation('relu'))
    model.add(Dense(1000))
    model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation('sigmoid'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, verbose=0)
    return model

In [552]:
%timeit -n 10 fit_dense()

10 loops, best of 3: 7.97 s per loop


In [558]:
%timeit -n 10 fit_circulant()

10 loops, best of 3: 3.31 s per loop


In [559]:
circulant_model = fit_circulant()

In [560]:
dense_model = fit_dense()

In [561]:
%timeit dense_model.predict(X_test)

10 loops, best of 3: 34.3 ms per loop


In [563]:
%timeit circulant_model.predict(X_test)

10 loops, best of 3: 23.8 ms per loop


In [564]:
def model_memory(model):
    return sum(w.nbytes for w in model.get_weights())

In [565]:
model_memory(dense_model)/model_memory(circulant_model)

76.80299785867237

In [567]:
K.eval(K.mean(keras.metrics.categorical_accuracy(K.variable(y_test), K.variable(dense_model.predict(X_test)))))

array(0.9781144857406616, dtype=float32)

In [568]:
K.eval(K.mean(keras.metrics.categorical_accuracy(K.variable(y_test), K.variable(circulant_model.predict(X_test)))))

array(0.9696969985961914, dtype=float32)